In [1]:
import requests
import pandas as pd
from pprint import pprint
import json
import datetime

In [23]:
team1 = 'Manchester United'
team2 = 'Liverpool'

In [129]:
header_params = {
            'x-rapidapi-key': "eff8f82f7dmsh5acc88f9a56c302p1b47acjsnfe16ac64e984",
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com"
            }

In [130]:
def get_team_id(team):    
    url = "https://api-football-v1.p.rapidapi.com/v3/teams"

    querystring = {"name":team}

    response = requests.request("GET", url, headers=header_params, params=querystring)

    data = json.loads(response.text)
    
    return data['response'][0]['team']['id']

In [131]:
team_id_1 = get_team_id(team1)
team_id_2 = get_team_id(team2)

In [25]:
url = "https://api-football-v1.p.rapidapi.com/v3/standings"

querystring = {"season":"2020","team":"33"}

response = requests.request("GET", url, headers=header_params, params=querystring)

print(response.text)

{"get":"standings","parameters":{"season":"2020","team":"33"},"errors":[],"results":2,"paging":{"current":1,"total":1},"response":[{"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\/\/media.api-sports.io\/football\/leagues\/39.png","flag":"https:\/\/media.api-sports.io\/flags\/gb.svg","season":2020,"standings":[[{"rank":2,"team":{"id":33,"name":"Manchester United","logo":"https:\/\/media.api-sports.io\/football\/teams\/33.png"},"points":74,"goalsDiff":29,"group":"Premier League","form":"WDLLW","status":"same","description":"Promotion - Champions League (Group Stage)","all":{"played":38,"win":21,"draw":11,"lose":6,"goals":{"for":73,"against":44}},"home":{"played":19,"win":9,"draw":4,"lose":6,"goals":{"for":38,"against":28}},"away":{"played":19,"win":12,"draw":7,"lose":0,"goals":{"for":35,"against":16}},"update":"2021-05-23T00:00:00+00:00"}]]}},{"league":{"id":2,"name":"UEFA Champions League","country":"World","logo":"https:\/\/media.api-sports.io\/football\/l

In [32]:
def get_standings(team_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/standings"
    
    year = datetime.datetime.now().year

    querystring = {"season":str(year-1),"team":str(team_id)}

    response = requests.request("GET", url, headers=header_params, params=querystring)
    
    data = json.loads(response.text)
    
    return data['response'][0]['league']['standings'][0][0]['rank']

In [27]:
get_standings(team_id_1)

2

In [28]:
def get_h2h(team_id_1, team_id_2):    
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/headtohead"
    
    team_id_1 = get_team_id(team1)
    team_id_2 = get_team_id(team2)
    
    h2h_string = str(team_id_1)+'-'+str(team_id_2)

    querystring = {"h2h":h2h_string}

    headers = header_params

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)

    df = pd.DataFrame([d['teams']['home']['name'], 
                       d['teams']['away']['name'],
                       d['fixture']['date'],
                       d['goals']['home'],
                       d['goals']['away']] for d in data['response'])
    df = df.rename(columns={0:'home_team', 1:'away_team', 2:'date', 3:'home_goals', 4:'away_goals'})
    
    return df

In [29]:
get_h2h(team1, team2)

,home_team,away_team,date,home_goals,away_goals
0,Manchester United,Liverpool,2019-02-24T14:05:00+00:00,0,0
1,Manchester United,Liverpool,2018-03-10T12:30:00+00:00,2,1
2,Manchester United,Liverpool,2017-01-15T16:00:00+00:00,1,1
3,Manchester United,Liverpool,2019-10-20T15:30:00+00:00,1,1
4,Manchester United,Liverpool,2015-09-12T16:30:00+00:00,3,1
5,Manchester United,Liverpool,2014-12-14T13:30:00+00:00,3,0
6,Manchester United,Liverpool,2014-03-16T13:30:00+00:00,0,3
7,Manchester United,Liverpool,2013-01-13T13:30:00+00:00,2,1
8,Manchester United,Liverpool,2012-02-11T12:45:00+00:00,2,1
9,Manchester United,Liverpool,2010-09-19T12:30:00+00:00,3,2


In [36]:
def get_injured_player2(fixture_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/injuries"

    querystring = {'fixture':str(fixture_id)}

    headers = header_params

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)

    df = pd.DataFrame([d['player']['id'], d['player_name']['name']] for d in data['response'])
    df = df.rename(columns={0:'player_id', 1:'player'})

    return df

In [67]:
def get_injured_player(team_id, date=datetime.date.today()):
    url = "https://api-football-v1.p.rapidapi.com/v3/injuries"

    querystring = {'date':str(date), 'team':str(team_id)}

    headers = header_params

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)

    df = pd.DataFrame([d['player']['id'], d['player']['name'], d['player']['type'], d['player']['photo']] for d in data['response'])
    df = df.rename(columns={0:'player_id', 1:'player', 2:'status', 3:'photo_link'})

    return df

In [68]:
get_injured_player('601', "2021-04-27")

,player_id,player,status,photo_link
0,38692,E. Palmer-Brown,Questionable,https://media.api-sports.io/football/players/3...
1,7704,C. Schosswendter,Questionable,https://media.api-sports.io/football/players/7...


In [84]:
def _get_last_fixture_ids(team_id, num_last_matches):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

    querystring = {"last":str(num_last_matches), 'team':str(team_id)}

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)
    
    fixture_ids = [f['fixture']['id'] for f in data['response']]

    return fixture_ids

In [85]:
_get_last_fixture_ids(team_id_1, 2)

[702510, 592875]

In [105]:
def get_starting_11(team_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/lineups"

    fixture_id = _get_last_fixture_ids(team_id, 1)

    querystring = {"team":str(team_id), 'fixture':str(fixture_id[0])}

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)
    
    df = pd.DataFrame([d['player']['id'], d['player']['name'], d['player']['pos'], d['player']['number']] for d in data['response'][0]['startXI'])
    df = df.rename(columns={0:'player_id', 1:'player', 2:'position', 3:'jersey_number'})

    return df

In [106]:
get_starting_11(463)

,player_id,player,position,jersey_number
0,6255,F. Assmann,G,35
1,26299,Emiliano Insúa,D,26
2,6404,J. Schunke,D,6
3,26735,Emanuel Insúa,D,13
4,6474,G. Gil,M,5
5,6550,F. Grahl,F,10
6,6211,G. Lodico,M,24
7,6212,L. Maciel,M,33
8,129900,J. Indacoechea,D,36
9,6126,F. Andrada,F,7


In [16]:
def get_next_fixture_id(team_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

    querystring = {"next":"1", 'team':str(team_id)}

    headers = header_params

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)  
    
    return data['response'][0]['fixture']['id']

In [17]:
get_next_fixture_id(team_id_1)

IndexError: list index out of range

In [118]:
def _get_fixture_id_h2h(team_id_1, team_id_2, next=True):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/headtohead"
    
    h2h_string = str(team_id_1)+'-'+str(team_id_2)

    if next==True:
        querystring = {"next":"1", 'h2h':h2h_string}
    else: 
        querystring = {"last":"1", 'h2h':h2h_string}

    response = requests.request("GET", url, headers=header_params, params=querystring)
    data = json.loads(response.text)  
    
    return data['response'][0]['fixture']['id']

In [119]:
get_fixture_id_h2h(team_id_1, team_id_2, next=False)

592831

In [124]:
def predictions(team_id_1, team_id_2):
    try:
        fixture_id = _get_fixture_id_h2h(team_id_1, team_id_2)
    except:
        print('using last fixture')
        fixture_id = _get_fixture_id_h2h(team_id_1, team_id_2, next=False)
    
    url = "https://api-football-v1.p.rapidapi.com/v3/predictions"

    querystring = {"fixture":fixture_id}

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = json.loads(response.text)

    return data['response'][0]['predictions']


In [125]:
predictions(team_id_1, team_id_2)

using last fixture


{'winner': {'id': 40, 'name': 'Liverpool', 'comment': 'Win or draw'},
 'win_or_draw': True,
 'under_over': None,
 'goals': {'home': '-2.5', 'away': '-2.5'},
 'advice': 'Double chance : draw or Liverpool',
 'percent': {'home': '10%', 'draw': '45%', 'away': '45%'}}